In [0]:
from pyspark.sql.functions import col, explode, flatten
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import json

In [0]:
errorfileurl = '/mnt/loksabhaattendance/loksabha_files/errors/errors.csv'

In [0]:
sessiondatesurl = '/mnt/loksabhaattendance/loksabha_files/flat_session_dates.csv'

sessiondates = spark.read.csv(sessiondatesurl, header=True)
list_sessiondates = [list(row) for row in sessiondates.collect()]
print(list_sessiondates)

[['13', '1', '1999/10/20'], ['13', '1', '1999/10/21'], ['13', '1', '1999/10/22'], ['13', '1', '1999/10/23'], ['13', '1', '1999/10/24'], ['13', '1', '1999/10/25'], ['13', '1', '1999/10/26'], ['13', '1', '1999/10/27'], ['13', '1', '1999/10/28'], ['13', '1', '1999/10/29'], ['13', '2', '1999/11/29'], ['13', '2', '1999/11/30'], ['13', '2', '1999/12/01'], ['13', '2', '1999/12/02'], ['13', '2', '1999/12/03'], ['13', '2', '1999/12/04'], ['13', '2', '1999/12/05'], ['13', '2', '1999/12/06'], ['13', '2', '1999/12/07'], ['13', '2', '1999/12/08'], ['13', '2', '1999/12/09'], ['13', '2', '1999/12/10'], ['13', '2', '1999/12/11'], ['13', '2', '1999/12/12'], ['13', '2', '1999/12/13'], ['13', '2', '1999/12/14'], ['13', '2', '1999/12/15'], ['13', '2', '1999/12/16'], ['13', '2', '1999/12/17'], ['13', '2', '1999/12/18'], ['13', '2', '1999/12/19'], ['13', '2', '1999/12/20'], ['13', '2', '1999/12/21'], ['13', '2', '1999/12/22'], ['13', '2', '1999/12/23'], ['13', '3', '2000/03/06'], ['13', '3', '2000/03/10'], 

In [0]:
errorschema = StructType ([
    StructField("loksabha", StringType(), True),
    StructField("sessionno", StringType(), True),
    StructField("date", StringType(), True)
])

In [0]:

errordf = spark.createDataFrame([], schema=errorschema)
for loksabha, sessionno, date in list_sessiondates:
    fileurl = '/mnt/loksabhaattendance/loksabha_files/loksabha=' + loksabha + '/session=' + sessionno + '/' + date.replace('/', '_') + '.csv'
    print('reading file ' + fileurl, end='\r')
    try:
        file = spark.read.csv(fileurl, header=True)
    except Exception as e:
        print('error reading file ' + fileurl)
        errordf = errordf.union(spark.createDataFrame([(loksabha, sessionno, date)], schema=errorschema))

error reading file /mnt/loksabhaattendance/loksabha_files/loksabha=13/session=5/2000_11_23.csv
error reading file /mnt/loksabhaattendance/loksabha_files/loksabha=13/session=7/2001_07_24.csv
reading f

*** WARNING: max output size exceeded, skipping output. ***

error reading file /mnt/loksabhaattendance/loksabha_files/loksabha=17/session=11/2023_03_15.csv
error reading file /mnt/loksabhaattendance/loksabha_files/loksabha=17/session=11/2023_03_28.csv
error reading file /mnt/loksabhaattendance/loksabha_files/loksabha=17/session=12/2023_07_28.csv


In [0]:
errordf.show()

+--------+---------+----------+
|loksabha|sessionno|      date|
+--------+---------+----------+
|      13|        5|2000/11/23|
|      13|        7|2001/07/24|
|      14|        9|2006/12/14|
|      15|       12|2012/11/23|
|      15|       13|2013/02/22|
|      15|       13|2013/03/12|
|      16|       17|2019/01/31|
|      17|       11|2023/03/15|
|      17|       11|2023/03/28|
|      17|       12|2023/07/28|
+--------+---------+----------+



In [0]:
errordf = errordf.withColumnRenamed('date', 'dates')
errordf.repartition(1).write.mode("overwrite").option("header",'true').csv(errorfileurl)